In [3]:
#! /usr/bin/python3
import os,sys
print("Extract Utility: Using osmium-tool ");

os.system("mason_packages/.link/bin/osmium --version | head -n 1");
print("");

Extract Utility: Using osmium-tool 



In [24]:
def get_bbox(path):
    print("Found: ",path)
    with open(path,'r') as file:
        return file.read()

In [31]:
def extract_bbox(path,name,found=False):
    if (found):
        bbox = get_bbox(path+"/"+name)
        command = "mason_packages/.link/bin/osmium extract --with-history --bbox={0} -o EXTRACTS/{1}.osh.pbf planet-history.latest.osm.pbf".format(bbox,name)
        print("Running command: \n",command)
        
    else:
        for p in os.listdir(path):
            if os.path.isdir(path+"/"+p):
                extract_bbox(path+"/"+p, name)
            elif p==name:
                return extract_bbox(path,name,found=True)

In [32]:
extract_bbox("bboxes",'detroit')

Found:  bboxes/USA/detroit
Running command: 
 mason_packages/.link/bin/osmium extract --with-history --bbox=-83.287956,42.255192,-82.910439,42.450233
 -o EXTRACTS/detroit.osh.pbf planet-history.latest.osm.pbf
